Importy

In [28]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

Wczytanie datasetu

In [29]:


pd.set_option('display.max_rows', 500)

df = pd.read_excel('../Analiza3/final_dataset_20-21.xlsx')

df.rename(columns=lambda x: x.replace(" (20/21)", ""), inplace=True)

df.head(), df.columns




(           Player                 Club  Age Position   Nation      Value  \
 0   Kylian Mbappe  Paris Saint-Germain   22   attack   France  144000000   
 1  Erling Haaland    Borussia Dortmund   21   attack   Norway  117000000   
 2      Harry Kane    Tottenham Hotspur   28   attack  England  108000000   
 3    Jadon Sancho    Manchester United   21   attack  England   90000000   
 4   Mohamed Salah         Liverpool FC   29   attack    Egypt   90000000   
 
   Contract Years Left          League      Squad    MP  ...  Offsides  \
 0                   1         Ligue 1  Paris S-G  31.0  ...      17.0   
 1                   3      Bundesliga   Dortmund  28.0  ...      15.0   
 2                   3  Premier League  Tottenham  35.0  ...      12.0   
 3                   5  Premier League   Dortmund  26.0  ...       9.0   
 4                   2  Premier League  Liverpool  37.0  ...      13.0   
 
    Crosses  Interceptions  Penalty Kicks Won  Penalties Conceded  Own Goals  \
 0     68.

Zamiana milionow na jednosci dla czytelnosci

In [30]:
df['Contract Years Left'] = pd.to_numeric(df['Contract Years Left'], errors='coerce')
df['Value'] = df['Value'] / 1000000

Oczyszczanie

In [31]:
# Check for missing values
missing_values = df.isnull().sum()

# Display columns with missing values
missing_columns = missing_values[missing_values > 0]
missing_columns


Contract Years Left                                                    45
Squad                                                                 165
MP                                                                    165
Starts                                                                165
Min                                                                   165
Total Mins/90                                                         165
Gls                                                                   165
Ast                                                                   165
Non-Penalty Goals                                                     165
Penalties Scored                                                      165
Penalties Attempted                                                   165
Yellow Cards                                                          165
Red Cards                                                             165
Gls/90                                

In [32]:
# Fill numeric columns with the median and categorical columns with the mode
for column in df.columns:
    if df[column].dtype in ['float64', 'int64']:
        df[column].fillna(df[column].median(), inplace=True)
    else:
        df[column].fillna(df[column].mode()[0], inplace=True)

# Verify that there are no more missing values
missing_values_after_cleaning = df.isnull().sum().sum()

# Display the total number of missing values after cleaning
missing_values_after_cleaning


0

In [33]:
# Checking data types
data_types = df.dtypes

# Convert categorical columns to category type
categorical_columns = df.select_dtypes(include=['object']).columns

for column in categorical_columns:
    df[column] = df[column].astype('category')

# Verify the data types after conversion
data_types_after_conversion = df.dtypes

data_types_after_conversion.head(20)  # Displaying first 20 to verify changes


Player                 category
Club                   category
Age                       int64
Position               category
Nation                 category
Value                   float64
Contract Years Left     float64
League                 category
Squad                  category
MP                      float64
Starts                  float64
Min                     float64
Total Mins/90           float64
Gls                     float64
Ast                     float64
Non-Penalty Goals       float64
Penalties Scored        float64
Penalties Attempted     float64
Yellow Cards            float64
Red Cards               float64
dtype: object

Podzial na grupy wedlug pozycji

In [34]:
# Podział danych na grupy według pozycji
df_attack = df[df['Position'] == 'attack']
df_midfield = df[df['Position'] == 'midfield']
df_defender = df[df['Position'] == 'Defender']
df_goalkeeper = df[df['Position'] == 'Goalkeeper']

Macierze korelacji dla kazdej z pozycji

In [35]:
# Funkcja do przefiltrowania nienumerycznych kolumn i obliczenia macierzy korelacji
def compute_numeric_correlation_matrix(df, target_variable='Value'):
    # Usunięcie nienumerycznych kolumn
    numeric_df = df.select_dtypes(include=['float64', 'int64'])
    correlation_matrix = numeric_df.corr()
    target_correlations = correlation_matrix[target_variable].sort_values(ascending=False)
    return target_correlations

# Macierze korelacji dla każdej pozycji z wykluczeniem kolumn nienumerycznych
correlation_attack_numeric = compute_numeric_correlation_matrix(df_attack)
correlation_midfield_numeric = compute_numeric_correlation_matrix(df_midfield)
correlation_defender_numeric = compute_numeric_correlation_matrix(df_defender)
correlation_goalkeeper_numeric = compute_numeric_correlation_matrix(df_goalkeeper)

# Wyświetlenie 10 cech najbardziej skorelowanych z wartością rynkową dla każdej pozycji
top_features_attack_numeric = correlation_attack_numeric.head(10)
top_features_midfield_numeric = correlation_midfield_numeric.head(10)
top_features_defender_numeric = correlation_defender_numeric.head(10)
top_features_goalkeeper_numeric = correlation_goalkeeper_numeric.head(10)

# Podpisanie wyników macierzy korelacji dla każdej pozycji
results = {
    "Attack": top_features_attack_numeric,
    "Midfield": top_features_midfield_numeric,
    "Defender": top_features_defender_numeric,
    "Goalkeeper": top_features_goalkeeper_numeric
}

for position, top_features in results.items():
    print(f'\n{position} top 10 features correlated with Value:\n{top_features}')





Attack top 10 features correlated with Value:
Value                                 1.000000
Carries into Attacking Penalty Box    0.649412
Non-penalty xG+ xA                    0.648306
Touches in Attacking Penalty Box      0.645765
Goal Creating Actions                 0.638570
Touches in Attacking 3rd              0.614356
Passes Leading to Goals               0.600124
Non-Penalty xG                        0.591531
Shot-Creating Actions                 0.591501
Passes Leading to Shot Attempt        0.587366
Name: Value, dtype: float64

Midfield top 10 features correlated with Value:
Value                                                   1.000000
Total Carries in Forward Direction                      0.557162
Passes Leading to Shot Attempt                          0.547952
Number of Times Player was Pass Target                  0.546497
Touches in Attacking 3rd                                0.544026
Number of Times Received Pass                           0.542396
Total Distance C

Regresja dla atakujacych

In [36]:
from sklearn.preprocessing import StandardScaler

# Wybór cech do modelu regresji liniowej dla atakujących
features_attack = [
    'Carries into Attacking Penalty Box',
    'Non-penalty xG+ xA',
    'Touches in Attacking Penalty Box',
    'Goal Creating Actions',
    'Touches in Attacking 3rd',
    'Passes Leading to Goals',
    'Non-Penalty xG',
    'Shot-Creating Actions',
    'Passes Leading to Shot Attempt'
]

# Przygotowanie danych
X_attack = df_attack[features_attack]
y_attack = df_attack['Value']

# Standaryzacja cech
scaler = StandardScaler()
X_attack_scaled = scaler.fit_transform(X_attack)

# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X_attack_scaled, y_attack, test_size=0.2, random_state=42)

# Tworzenie modelu regresji liniowej
model = LinearRegression()
model.fit(X_train, y_train)

# Predykcja
y_pred = model.predict(X_test)

# Ocena modelu
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Wagi cech w modelu
feature_weights = pd.Series(model.coef_, index=features_attack)

print(f'\nAttack MSE: {mse}')
print(f'\nAttack R2: {r2}')
print(f'\nAttack feature weights:\n{feature_weights}')



Attack MSE: 231.42830196548624

Attack R2: 0.5501036070421406

Attack feature weights:
Carries into Attacking Penalty Box    5.686843
Non-penalty xG+ xA                    3.966941
Touches in Attacking Penalty Box     -1.243526
Goal Creating Actions                 2.241621
Touches in Attacking 3rd              3.938172
Passes Leading to Goals               3.854657
Non-Penalty xG                        1.971747
Shot-Creating Actions                -5.172144
Passes Leading to Shot Attempt       -1.421304
dtype: float64


VIF

In [37]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Obliczanie wartości VIF dla zmiennych niezależnych
X_vif = df_attack[features_attack]
X_vif_scaled = scaler.fit_transform(X_vif)

# Tworzenie DataFrame dla wartości VIF
vif_data = pd.DataFrame()
vif_data["Feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif_scaled, i) for i in range(X_vif_scaled.shape[1])]

vif_data.sort_values(by="VIF", ascending=False)

,Feature,VIF
1,Non-penalty xG+ xA,57.133330
7,Shot-Creating Actions,36.642737
6,Non-Penalty xG,35.256272
8,Passes Leading to Shot Attempt,27.755390
3,Goal Creating Actions,15.771820
2,Touches in Attacking Penalty Box,14.509917
4,Touches in Attacking 3rd,13.944028
5,Passes Leading to Goals,12.835166
0,Carries into Attacking Penalty Box,3.765694


In [39]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV

# Definiowanie modelu Ridge Regression z walidacją krzyżową
ridge = Ridge()
parameters_ridge = {'alpha': [0.1, 1.0, 10.0, 100.0, 1000.0]}
ridge_regressor = GridSearchCV(ridge, parameters_ridge, cv=5)
ridge_regressor.fit(X_train, y_train)

# Predykcja Ridge Regression
y_pred_ridge = ridge_regressor.predict(X_test)

# Ocena modelu Ridge Regression
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

# Najlepszy parametr alpha dla Ridge Regression
best_alpha_ridge = ridge_regressor.best_params_

# Wagi cech w modelu Ridge Regression
feature_weights_ridge = pd.Series(ridge_regressor.best_estimator_.coef_, index=features_attack)

# Wyniki Ridge Regression
ridge_results = {
    "mse": mse_ridge,
    "r2": r2_ridge,
    "best_alpha": best_alpha_ridge,
    "feature_weights": feature_weights_ridge
}

# Definiowanie modelu Lasso Regression z walidacją krzyżową
lasso = Lasso()
parameters_lasso = {'alpha': [0.1, 1.0, 10.0, 100.0, 1000.0]}
lasso_regressor = GridSearchCV(lasso, parameters_lasso, cv=5)
lasso_regressor.fit(X_train, y_train)

# Predykcja Lasso Regression
y_pred_lasso = lasso_regressor.predict(X_test)

# Ocena modelu Lasso Regression
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

# Najlepszy parametr alpha dla Lasso Regression
best_alpha_lasso = lasso_regressor.best_params_

# Wagi cech w modelu Lasso Regression
feature_weights_lasso = pd.Series(lasso_regressor.best_estimator_.coef_, index=features_attack)

# Wyniki Lasso Regression
lasso_results = {
    "mse": mse_lasso,
    "r2": r2_lasso,
    "best_alpha": best_alpha_lasso,
    "feature_weights": feature_weights_lasso
}

ridge_results, lasso_results


({'mse': 295.32202096920605,
  'r2': 0.4258942797113652,
  'best_alpha': {'alpha': 1000.0},
  'feature_weights': Carries into Attacking Penalty Box    1.376538
  Non-penalty xG+ xA                    1.192678
  Touches in Attacking Penalty Box      1.156169
  Goal Creating Actions                 1.142656
  Touches in Attacking 3rd              0.986136
  Passes Leading to Goals               1.123377
  Non-Penalty xG                        1.135622
  Shot-Creating Actions                 0.828249
  Passes Leading to Shot Attempt        0.826112
  dtype: float64},
 {'mse': 250.59246679366865,
  'r2': 0.5128484893360314,
  'best_alpha': {'alpha': 1.0},
  'feature_weights': Carries into Attacking Penalty Box    4.604402
  Non-penalty xG+ xA                    2.324947
  Touches in Attacking Penalty Box      0.000000
  Goal Creating Actions                 0.715147
  Touches in Attacking 3rd              0.000000
  Passes Leading to Goals               2.804053
  Non-Penalty xG           